In [48]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import date
from datetime import datetime
from pandas_profiling import ProfileReport

# Data Wrangling Exercise 

In this exercise, I will work on wrangling the two kickstarter datasets that I submitted in my proposal. I have combined the two datasets to get a dataset with more observations. 

The question that I seek to answer is: How can Kickstarter increase the success rate of its projects by 10% over the next year by prioritizing to advertise those projects with the likelihood of higher success? 

### Loading/Exploring datasets

In [49]:
#Loading/reading the first dataset
ks_1 = pd.read_csv("ks-projects-201801.csv")

In [50]:
ks_1.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [51]:
#Loading/reading the second dataset
ks_2 = pd.read_csv("Kickstarter_projects_Feb19.csv")

In [52]:
ks_2.head()

,id,name,currency,main_category,sub_category,launched_at,deadline,duration,goal_usd,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018-10-30 20:00:02,2018-11-15 17:59:00,16.0,2000.00000,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.00000
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018-08-06 10:00:43,2018-09-05 10:00:43,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.50512
2,454186436,"Live Printing with SX8: ""Squeegee Pulp Up""",USD,fashion,Apparel,2017-06-09 15:41:03,2017-07-09 15:41:03,30.0,1100.00000,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.00000
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014-09-25 18:46:01,2014-11-10 06:00:00,45.0,3500.00000,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.00000
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016-11-28 16:35:11,2017-01-27 16:35:11,60.0,30000.00000,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.00000


In [53]:
#Renaming column names from the first dataset so that concating is easier

ks_1.rename({'ID': 'id', 'category':'sub_category', 'state':'status', 'usd_pledged_real': 'usd_pledged', 'usd_goal_real':'usd_goal'}, axis=1, inplace = True)


In [54]:
#Finding the range of years in the first dataset
ks_1.head()
ks_1['launched'].min()


'1970-01-01 01:00:00'

In [55]:
ks_1['launched'].max()

'2018-01-02 15:02:31'

We find that the first dataset, ks_1, has observations from 1970 - 2018. 

In [56]:
# Renaming columns from the second dataset so that concatenating is easier. 
ks_2.rename({'launched_at': 'launched', 'goal_usd':'usd_goal'}, axis=1, inplace = True)
ks_2.head()

,id,name,currency,main_category,sub_category,launched,deadline,duration,usd_goal,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018-10-30 20:00:02,2018-11-15 17:59:00,16.0,2000.00000,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.00000
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018-08-06 10:00:43,2018-09-05 10:00:43,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.50512
2,454186436,"Live Printing with SX8: ""Squeegee Pulp Up""",USD,fashion,Apparel,2017-06-09 15:41:03,2017-07-09 15:41:03,30.0,1100.00000,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.00000
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014-09-25 18:46:01,2014-11-10 06:00:00,45.0,3500.00000,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.00000
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016-11-28 16:35:11,2017-01-27 16:35:11,60.0,30000.00000,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.00000


In [57]:
#Finding the range of years in the second dataset
ks_2['launched'].min()

'2009-04-28 11:55:41'

In [58]:
ks_2['launched'].max()

'2019-02-08 03:40:11'

We find that the second dataset, ks_2, has observations from 2009-2019. 

In [59]:
#Finding out the number of rows from the first and the second dataset. 
ks_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192548 entries, 0 to 192547
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             192548 non-null  int64  
 1   name           192548 non-null  object 
 2   currency       192548 non-null  object 
 3   main_category  192548 non-null  object 
 4   sub_category   192548 non-null  object 
 5   launched       192548 non-null  object 
 6   deadline       192548 non-null  object 
 7   duration       192548 non-null  float64
 8   usd_goal       192548 non-null  float64
 9   city           192548 non-null  object 
 10  state          192548 non-null  object 
 11  country        192548 non-null  object 
 12  blurb_length   192548 non-null  int64  
 13  name_length    192548 non-null  int64  
 14  status         192548 non-null  object 
 15  start_month    192548 non-null  int64  
 16  end_month      192548 non-null  int64  
 17  start_Q        192548 non-nul

In [60]:
ks_1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             378661 non-null  int64  
 1   name           378657 non-null  object 
 2   sub_category   378661 non-null  object 
 3   main_category  378661 non-null  object 
 4   currency       378661 non-null  object 
 5   deadline       378661 non-null  object 
 6   goal           378661 non-null  float64
 7   launched       378661 non-null  object 
 8   pledged        378661 non-null  float64
 9   status         378661 non-null  object 
 10  backers        378661 non-null  int64  
 11  country        378661 non-null  object 
 12  usd pledged    374864 non-null  float64
 13  usd_pledged    378661 non-null  float64
 14  usd_goal       378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


We see that ks_1 has 378661 rows and ks_2 has 192548 rows. 

### Joining Datasets

In [61]:
#concatenating datasets
frames = [ks_1, ks_2]
kickstarter = pd.concat(frames)

In [62]:
# quickly understanding the datasets
kickstarter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571209 entries, 0 to 192547
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             571209 non-null  int64  
 1   name           571205 non-null  object 
 2   sub_category   571209 non-null  object 
 3   main_category  571209 non-null  object 
 4   currency       571209 non-null  object 
 5   deadline       571209 non-null  object 
 6   goal           378661 non-null  float64
 7   launched       571209 non-null  object 
 8   pledged        378661 non-null  float64
 9   status         571209 non-null  object 
 10  backers        378661 non-null  float64
 11  country        571209 non-null  object 
 12  usd pledged    374864 non-null  float64
 13  usd_pledged    571209 non-null  float64
 14  usd_goal       571209 non-null  float64
 15  duration       192548 non-null  float64
 16  city           192548 non-null  object 
 17  state          192548 non-nul

### Cleaning the data

It is essential to check whether while combining the datasets, the IDs have been duplicated. These are unique kickstarter IDs that identify each individual project. 

In [64]:
ids = kickstarter['id']
kickstarter[ids.isin(ids[ids.duplicated()])].sort_values('id')

,id,name,sub_category,main_category,currency,deadline,goal,launched,pledged,status,...,usd_goal,duration,city,state,blurb_length,name_length,start_month,end_month,start_Q,end_Q
123397,18520,Grandma's are Life,World Music,music,USD,2016-11-18 14:32:40,NaN,2016-10-19 13:32:40,NaN,failed,...,15000.00000,30.0,Cleveland,OH,24.0,3.0,10.0,11.0,Q4,Q4
167485,18520,Grandma's are Life,World Music,Music,USD,2016-11-18,15000.0,2016-10-19 15:32:40,62.0,failed,...,15000.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38972,21109,Meta,Performance Art,art,GBP,2015-05-06 21:00:00,NaN,2015-04-07 22:37:44,NaN,successful,...,192.97370,29.0,Dundee,Scotland,24.0,1.0,4.0,5.0,Q2,Q2
218065,21109,Meta,Performance Art,Art,GBP,2015-05-06,150.0,2015-04-08 00:37:44,173.0,successful,...,231.20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189966,24380,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,food,USD,2015-11-26 16:25:33,NaN,2015-10-27 15:25:33,NaN,failed,...,20000.00000,30.0,61704,IL,24.0,9.0,10.0,11.0,Q4,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86549,2147460119,Moments of Insight,Conceptual Art,art,USD,2011-08-04 04:30:00,NaN,2011-06-24 19:29:15,NaN,successful,...,18000.00000,40.0,Bozeman,MT,21.0,3.0,6.0,8.0,Q2,Q3
192410,2147466649,Dream Wedding Photo's,Fine Art,photography,CAD,2014-07-24 15:23:00,NaN,2014-07-11 18:56:05,NaN,failed,...,1132.82955,13.0,Halifax,NS,22.0,3.0,7.0,7.0,Q3,Q3
225210,2147466649,Dream Wedding Photo's,Fine Art,Photography,CAD,2014-07-24,1500.0,2014-07-11 20:56:05,0.0,failed,...,1393.02000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225212,2147476221,DrinkTanks Offers the World's Largest Growler ...,Drinks,Food,USD,2015-04-10,75000.0,2015-03-02 16:14:38,304142.0,successful,...,75000.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that there are some duplicated IDs, which we will drop as these are the same projects. 

In [65]:
kickstarter.drop_duplicates(subset=['id'], keep='first', inplace = True)

In [66]:
len(kickstarter)

408278

Next, we will set the range of years we are interested in analyzing. The first dataset had data from 1970 which may not be pertinent to our question. Therefore, we choose data from 2009-2019 to make the data recent and relevant to our query. 

In [67]:
#Applying datetime so that we can filter the projects between 2009-2019. 
kickstarter['launched_dt'] = kickstarter['launched'].apply(pd.to_datetime)
kickstarter['deadline_dt'] = kickstarter['deadline'].apply(pd.to_datetime)


In [68]:
kickstarter.head()

,id,name,sub_category,main_category,currency,deadline,goal,launched,pledged,status,...,city,state,blurb_length,name_length,start_month,end_month,start_Q,end_Q,launched_dt,deadline_dt
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-11 12:12:28,2015-10-09
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-02 04:43:57,2017-11-01
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-01-12 00:20:50,2013-02-26
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-17 03:24:11,2012-04-16
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-04 08:35:03,2015-08-29


In [69]:
#Checking to see whether the datetime object is as expected. 
kickstarter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408278 entries, 0 to 192542
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             408278 non-null  int64         
 1   name           408274 non-null  object        
 2   sub_category   408278 non-null  object        
 3   main_category  408278 non-null  object        
 4   currency       408278 non-null  object        
 5   deadline       408278 non-null  object        
 6   goal           378661 non-null  float64       
 7   launched       408278 non-null  object        
 8   pledged        378661 non-null  float64       
 9   status         408278 non-null  object        
 10  backers        378661 non-null  float64       
 11  country        408278 non-null  object        
 12  usd pledged    374864 non-null  float64       
 13  usd_pledged    408278 non-null  float64       
 14  usd_goal       408278 non-null  float64       
 15  

In [70]:
#Extracting the year out of the 'launched_year' and 'deadline_year' column

kickstarter['launched_year'] = kickstarter['launched_dt'].dt.year

kickstarter['deadline_year'] = kickstarter['deadline_dt'].dt.year

In [71]:
#checking to see if the 'launched_year' and 'deadline_year' columns are integers
kickstarter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408278 entries, 0 to 192542
Data columns (total 28 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             408278 non-null  int64         
 1   name           408274 non-null  object        
 2   sub_category   408278 non-null  object        
 3   main_category  408278 non-null  object        
 4   currency       408278 non-null  object        
 5   deadline       408278 non-null  object        
 6   goal           378661 non-null  float64       
 7   launched       408278 non-null  object        
 8   pledged        378661 non-null  float64       
 9   status         408278 non-null  object        
 10  backers        378661 non-null  float64       
 11  country        408278 non-null  object        
 12  usd pledged    374864 non-null  float64       
 13  usd_pledged    408278 non-null  float64       
 14  usd_goal       408278 non-null  float64       
 15  

In [72]:
#filtering for those projects that were launched on or after 2009. 
df_ks = kickstarter[(kickstarter['launched_year'] >=2009) & (kickstarter['deadline_year'] >= 2009)]

In [73]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408271 entries, 0 to 192542
Data columns (total 28 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             408271 non-null  int64         
 1   name           408267 non-null  object        
 2   sub_category   408271 non-null  object        
 3   main_category  408271 non-null  object        
 4   currency       408271 non-null  object        
 5   deadline       408271 non-null  object        
 6   goal           378654 non-null  float64       
 7   launched       408271 non-null  object        
 8   pledged        378654 non-null  float64       
 9   status         408271 non-null  object        
 10  backers        378654 non-null  float64       
 11  country        408271 non-null  object        
 12  usd pledged    374857 non-null  float64       
 13  usd_pledged    408271 non-null  float64       
 14  usd_goal       408271 non-null  float64       
 15  

In [74]:
#checking out a sample 
df_ks.sample(20)

,id,name,sub_category,main_category,currency,deadline,goal,launched,pledged,status,...,blurb_length,name_length,start_month,end_month,start_Q,end_Q,launched_dt,deadline_dt,launched_year,deadline_year
18008,1091387726,Glory Gold Shoes,Footwear,Fashion,USD,2014-11-10,20000.0,2014-09-25 03:00:37,1400.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2014-09-25 03:00:37,2014-11-10 00:00:00,2014,2014
98428,1499639619,Alaska Seafood Source: Sustainable & Wild Seaf...,Food,Food,USD,2014-04-19,60000.0,2014-03-18 22:58:01,22381.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-18 22:58:01,2014-04-19 00:00:00,2014,2014
25273,1128362883,One A Day For A Week,Shorts,Film & Video,USD,2013-09-22,2200.0,2013-08-23 05:19:51,2706.0,successful,...,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-23 05:19:51,2013-09-22 00:00:00,2013,2013
148239,1753502649,Just Playing With Jason,Webseries,Film & Video,USD,2012-11-15,2500.0,2012-10-16 21:03:39,130.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2012-10-16 21:03:39,2012-11-15 00:00:00,2012,2012
208926,2064652720,we were too far gone to notice: 2017 calendar,Calendars,Publishing,USD,2017-02-03,150.0,2017-01-22 02:30:22,365.0,successful,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-22 02:30:22,2017-02-03 00:00:00,2017,2017
130532,1662715162,Bards of Bywater tribute to Leonard Cohen CONC...,Jazz,Music,USD,2012-03-16,16000.0,2012-02-23 20:58:34,3731.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2012-02-23 20:58:34,2012-03-16 00:00:00,2012,2012
182157,1927282821,Print Release of (re)Visions: Alice,Fiction,Publishing,USD,2011-10-12,350.0,2011-09-08 03:17:30,1587.0,successful,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-08 03:17:30,2011-10-12 00:00:00,2011,2011
348621,846182948,John Denver Tribute Songs By Gary Reynolds CD,Country & Folk,Music,USD,2017-11-16,600.0,2017-10-18 20:09:30,169.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-10-18 20:09:30,2017-11-16 00:00:00,2017,2017
339384,798770629,Nightwatch,Webseries,Film & Video,USD,2013-09-09,7000.0,2013-08-11 06:25:02,2446.0,failed,...,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-11 06:25:02,2013-09-09 00:00:00,2013,2013
188908,1961228888,Amsterdam Killings. (Canceled),Documentary,Film & Video,EUR,2015-07-23,7000.0,2015-06-23 23:29:11,0.0,canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-23 23:29:11,2015-07-23 00:00:00,2015,2015


In [75]:
#dropping columns that are not of analytical importance. These columns were in one dataset and not in the other. Therefore they had a null values when concatenating. 
#It was not possible to fill these columns with any values like average values. Thus, deciding to drop it. 
df_ks.drop(columns = ['city', 'state', 'blurb_length', 'name_length', 'start_month', 'end_month', 'start_Q', 'end_Q', 'usd pledged', 'pledged', 'backers', 'goal'], inplace = True)

/Users/shikshya/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [76]:
#checking the count and percentage of null values 
missing = pd.concat([df_ks.isnull().sum(), 100 * df_ks.isnull().sum()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count', '%'], ascending = True)

,count,%
id,0,0
sub_category,0,0
main_category,0,0
currency,0,0
deadline,0,0
launched,0,0
status,0,0
country,0,0
usd_pledged,0,0
usd_goal,0,0


In [36]:
df_ks[df_ks['name'].isnull()]

,id,name,sub_category,main_category,currency,deadline,launched,status,country,usd_pledged,usd_goal,duration,launched_dt,deadline_dt,launched_year,deadline_year
166851,1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,2012-01-01 12:35:31,failed,US,100.00,200000.00,NaN,2012-01-01 12:35:31,2012-02-29,2012,2012
307234,634871725,NaN,Video Games,Games,GBP,2013-01-06,2012-12-19 23:57:48,failed,GB,316.05,3224.97,NaN,2012-12-19 23:57:48,2013-01-06,2012,2013
309991,648853978,NaN,Product Design,Design,USD,2016-07-18,2016-06-18 05:01:47,suspended,US,0.00,2500.00,NaN,2016-06-18 05:01:47,2016-07-18,2016,2016
338931,796533179,NaN,Painting,Art,USD,2011-12-05,2011-11-06 23:55:55,failed,US,220.00,35000.00,NaN,2011-11-06 23:55:55,2011-12-05,2011,2011


We see that name has 4 null values. However, the name of the project is of less importance as they have unique IDs associated with them. Therefore, we chose to leave it as is. 

The 'duration' column is an interesting one. Duration may be of analytical importance to predict the success, failure, amount pledged, and goal of the project. The second dataset had duration values whereas the first dataset did not. Therefore, it is necessary to calculate the duration for the following project. 

In [77]:
small = df_ks[['duration', 'launched_dt', 'deadline_dt']]
list_1 = []
for idx, row in small.iterrows():
    if pd.isnull(row[0]):
        time_delta = row[2] - row[1]
        list_1.append(time_delta.days) #extracting the days out of the time delta.
    else:
        list_1.append(int(row[0]))

In [78]:
df_ks['duration'] = list_1

<ipython-input-78-3749902b8086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ks['duration'] = list_1


In [79]:
df_ks.head()

,id,name,sub_category,main_category,currency,deadline,launched,status,country,usd_pledged,usd_goal,duration,launched_dt,deadline_dt,launched_year,deadline_year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,0.0,1533.95,58,2015-08-11 12:12:28,2015-10-09,2015,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,US,2421.0,30000.00,59,2017-09-02 04:43:57,2017-11-01,2017,2017
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,US,220.0,45000.00,44,2013-01-12 00:20:50,2013-02-26,2013,2013
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,US,1.0,5000.00,29,2012-03-17 03:24:11,2012-04-16,2012,2012
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,canceled,US,1283.0,19500.00,55,2015-07-04 08:35:03,2015-08-29,2015,2015


In [80]:
#checking again for null values
missing = pd.concat([df_ks.isnull().sum(), 100 * df_ks.isnull().sum()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count', '%'], ascending = True)

,count,%
id,0,0
sub_category,0,0
main_category,0,0
currency,0,0
deadline,0,0
launched,0,0
status,0,0
country,0,0
usd_pledged,0,0
usd_goal,0,0


In [40]:
#checking again to see the data types of columns is what we want it to be. 
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408271 entries, 0 to 192542
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             408271 non-null  int64         
 1   name           408267 non-null  object        
 2   sub_category   408271 non-null  object        
 3   main_category  408271 non-null  object        
 4   currency       408271 non-null  object        
 5   deadline       408271 non-null  object        
 6   launched       408271 non-null  object        
 7   status         408271 non-null  object        
 8   country        408271 non-null  object        
 9   usd_pledged    408271 non-null  float64       
 10  usd_goal       408271 non-null  float64       
 11  duration       408271 non-null  int64         
 12  launched_dt    408271 non-null  datetime64[ns]
 13  deadline_dt    408271 non-null  datetime64[ns]
 14  launched_year  408271 non-null  int64         
 15  

In [85]:
#summary statistics of all the numerical columns
df_ks.describe()

,id,usd_pledged,usd_goal,duration,launched_year,deadline_year
count,4.082710e+05,4.082710e+05,4.082710e+05,408271.000000,408271.000000,408271.000000
mean,1.074309e+09,9.783897e+03,4.354569e+04,33.104969,2014.521930,2014.592751
std,6.193319e+08,9.230929e+04,1.113579e+06,12.716384,2.095593,2.094641
min,5.971000e+03,0.000000e+00,1.000000e-02,0.000000,2009.000000,2009.000000
25%,5.373578e+08,3.700000e+01,2.000000e+03,29.000000,2013.000000,2013.000000
50%,1.074942e+09,7.000000e+02,5.200000e+03,29.000000,2015.000000,2015.000000
75%,1.610296e+09,4.350005e+03,1.500000e+04,36.000000,2016.000000,2016.000000
max,2.147476e+09,2.033899e+07,1.663614e+08,93.000000,2019.000000,2019.000000


Describing Data to get a visual and numerical description of the columns. 

In [86]:
profile = ProfileReport(df_ks, title="Pandas Profiling Report", minimal=True)

In [87]:
profile.to_file("your_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

We see that there are more failed projects than successful ones. We also see that the majority of the projects are in the US. The average duration of the projects are thirty days. We see that some pledged and goal amounts are zero, the effects of which may be interesting to explore in the exploratory data analysis section. 